## Imports

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=FutureWarning)

%matplotlib inline
import pandas as pd
import numpy as np
import sys, os, time, datetime
import matplotlib.pyplot as plt


sys.path.append('libraries')
import signal_processing as spl 
import prediction_lib as pl
pl = reload(pl)
spl = reload(spl)

In [2]:
import numpy as np
from bs4 import BeautifulSoup
from collections import Counter
import urllib
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
import matplotlib.colors as colors
import pandas as pd
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.linear_model import LogisticRegressionCV as LogRegCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.decomposition import PCA
from tqdm import tqdm
import sys
from scipy import optimize
from IPython import display
from collections import Counter
from sklearn.externals import joblib

from scipy.cluster.vq import kmeans2,vq, whiten
from sklearn.cluster import KMeans
from hmmlearn.hmm import MultinomialHMM
from python_speech_features import mfcc

##  Load in Signal DF

In [3]:
load = True
name = 'perfect_dataset'
INPUT_FILE = 'data/%s.csv' % name
out_name = 'results/%s_RESULTS.csv' % name

In [4]:
if load and os.path.exists(out_name):
    RESULTS_DF = pd.read_csv(out_name, index_col = 0)
    RESULTS_DF.head()
else:
    RESULTS_DF = None


,ACCURACY_WO_SPACES,DO_ALL,INPUT_FILE,MAX_COLS,MFCC_END,MFCC_START,MODEL,MODEL_FNAME,NUM_CLUSTERS,N_COMPONENTS,OVERALL_ACCURACY,PREDICTION,RATE,SCORE,SMOOTH,TOL,WHITEN
2016-12-02 18:49:05.655758,0.232356,False,data/perfect_dataset.csv,-1,-60.0,0,"MultinomialHMM(algorithm='viterbi', init_param...",models/2016-12-02 18:49:05.655758-0.2936576889...,50,100.0,0.293658,athee wad ledingrnd tl and st t anked in s thi...,44100,-13707.036373,54.598150,0.01,True
59:31.7,0.223308,False,data/perfect_dataset.csv,-1,NaN,0,"MultinomialHMM(algorithm='viterbi', init_param...",NaN,50,100.0,0.282653,idnis sas hetindund ts hes edat inded is dindi...,44100,-13722.486180,2.718282,0.01,False
2016-12-02 18:26:08.172422,0.210279,False,data/perfect_dataset.csv,-1,-60.0,0,"MultinomialHMM(algorithm='viterbi', init_param...",models/2016-12-02 18:26:08.172422-0.2823631624...,50,100.0,0.282363,adive wan hedine ng tiched thed tiche in aithi...,44100,-13453.805850,7.389056,0.01,True
53:09.3,0.218241,False,data/perfect_dataset.csv,-1,NaN,0,"MultinomialHMM(algorithm='viterbi', init_param...",NaN,50,100.0,0.279757,isnis sas hetindund st hes ksct hived it windi...,44100,-13591.706470,0.049787,0.01,False
2016-12-02 18:42:08.082977,0.205936,False,data/perfect_dataset.csv,-1,-60.0,0,"MultinomialHMM(algorithm='viterbi', init_param...",models/2016-12-02 18:42:08.082977-0.2742542716...,50,100.0,0.274254,athee the cedingune ss hed as s anghe is and i...,44100,-13584.794944,1.000000,0.01,True


In [5]:
char_inps_raw = pd.read_csv(INPUT_FILE, index_col=0)
char_inps_raw['char'] = char_inps_raw['char'].fillna(' ').replace('#SPACE', ' ')
# char_inps_raw['peak_diff'] =char_inps_raw['peak_diff'].fillna(0)
char_inps_raw = char_inps_raw.fillna(0)
char_inps_raw.tail(10)

,peak time,char,start time,end time,signal,0,1,2,3,4,...,15425,15426,15427,15428,15429,15430,15431,15432,15433,15434
3443,1205.164819,,1205.129819,1205.479819,0.000357,-0.100998,-0.015492,0.084944,0.022892,0.118922,...,0.988503,0.888722,1.186432,1.165176,0.144874,0.252931,0.142511,0.196231,1.019574,0.243095
3444,1205.536387,o,1205.501387,1205.851387,0.000357,0.605787,0.673882,0.841651,0.465785,0.397254,...,-3.536588,5.058556,5.898054,6.116207,6.306228,-10.719934,5.384215,18.141334,-3.398774,4.310902
3445,1205.602626,u,1205.567626,1205.917626,0.000357,0.370805,0.367855,0.193740,0.271778,0.217057,...,-0.052352,0.434728,0.559545,-0.040595,-0.744939,-0.705834,-0.345340,-0.434586,-0.457719,-0.302066
3446,1205.869425,t,1205.834425,1206.184425,0.000357,0.574571,0.536336,0.310582,0.287124,0.468396,...,7.602115,-4.575600,1.406101,-2.978368,1.717977,11.550069,-4.738052,-4.291237,-3.474019,-3.132150
3447,1206.152522,,1206.117522,1206.467522,0.000357,0.431744,0.391007,0.554542,0.688468,0.681929,...,-1.686501,-1.405832,-1.132361,-1.400418,-1.635337,-1.081181,-0.747627,-0.437727,-0.078946,-0.597740
3448,1206.603440,a,1206.568440,1206.918440,0.000357,-0.032560,0.000937,-0.072859,-0.052033,-0.111156,...,-0.053287,0.107539,0.242174,0.231995,0.234771,0.165148,0.217436,0.225331,-0.098057,-0.262253
3449,1206.880633,g,1206.845633,1207.195633,0.000357,0.993742,2.423808,1.609095,3.906103,-1.916781,...,-0.214781,-0.203393,-0.293734,-0.352500,-0.279817,-0.146706,-0.194231,-0.147599,-0.075321,-0.133153
3450,1207.134864,a,1207.099864,1207.449864,0.000357,-0.241887,-0.305712,-0.413097,-0.455908,-0.563319,...,-3.410009,3.432781,1.741468,1.822611,1.289886,-0.678719,-1.402292,2.042433,3.062629,0.586512
3451,1207.493027,i,1207.458027,1207.808027,0.000357,1.338969,-2.046932,-1.036754,1.714020,-0.265125,...,0.044424,-0.199461,-0.242156,0.118358,0.091515,-0.106329,-0.103860,-0.028796,-0.180932,-0.132623
3452,1207.817996,n,1207.782996,1208.132996,0.000357,-0.326185,-0.451966,-0.480052,-0.572480,-0.523397,...,0.584964,0.403560,0.429286,0.358761,0.257468,0.414053,0.427385,0.309221,0.406631,0.378481


In [6]:
targ_s = ''.join(char_inps_raw['char'].values)
targ_s

'alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book without pictures or conversations so she was considering in her own mind as well as she could for the day made her feel very sleepy and stupid whether the pleasure of making a daisy chai would be worth the trouble of getting up and picking the daisies when sudenly a white rabbit with pink eyes ran close by her there was nothing as remarkable in that nor did alice think it so very much out of the way to hear the rabbit say to itself oh dear oh dear i shall be too late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it all seemed quite natural but when the rabbit actually took a watch out of its pocket and burning with curiosity she ran across the field after it and was just about to 

In [7]:
PARAM_RESULTS ={}

## Processing 

In [8]:
RATE = 44100
WHITEN = True
SAVE_THRESH= 0.25
TOL = 1e-2

should_loop = True


MAX_COLS = -1
MFCC_START = [0]
MFCC_END = [-60]
NUM_CLUSTERS = [50]
N_COMPONENTS = [100]
DO_ALL = False


REPEATS = 4
SMOOTHS = map(lambda x : np.exp(x), range(0,6))

In [14]:
def build_params(smooth, end, start, cluster, comp):
    _id = datetime.datetime.now()
    PARAM_RESULTS[_id] = {'RATE' : RATE,
        'TOL' : TOL,
        'INPUT_FILE' : INPUT_FILE,
         'MAX_COLS': MAX_COLS,
          'NUM_CLUSTERS' : cluster,
          'N_COMPONENTS' : comp,
          'MFCC_START' : start,
          'MFCC_END' : end,
          'WHITEN' : WHITEN,
          'DO_ALL' : DO_ALL,
          'SMOOTH' : smooth,
          'PREDICTION' : None,
          'ACCURACY_WO_SPACES' : None,
         'OVERALL_ACCURACY' : None,
         'SCORE' : None,
         'MODEL' : None,
        'MODEL_FNAME' : None
                          
         }
    return PARAM_RESULTS[_id], _id

def save(model = None, accuracy = None, _id = None):
    global RESULTS_DF
    if model:
        PARAM_RESULTS[_id]['MODEL_FNAME'] = "models/%s-%s.pkl" % (_id, accuracy)
        joblib.dump(model, "models/%s-%s.pkl" % (_id, accuracy))
        
    if RESULTS_DF is not None:
        RESULTS_DF = RESULTS_DF.append(pd.DataFrame(PARAM_RESULTS).T).drop_duplicates()
    else:
        RESULTS_DF = pd.DataFrame(PARAM_RESULTS).T
    
    
    
    
    RESULTS_DF.sort_values(['OVERALL_ACCURACY', 'ACCURACY_WO_SPACES'], ascending=False).to_csv(out_name)
    
    by_smooth = RESULTS_DF[['ACCURACY_WO_SPACES','OVERALL_ACCURACY', 'SMOOTH']].dropna().astype(float).groupby('SMOOTH')
#     print 'max'
#     display.display(by_smooth.max().sort_values(['OVERALL_ACCURACY', 'ACCURACY_WO_SPACES'], ascending =False).head(3))
#     print 'mean'
#     display.display(by_smooth.mean().sort_values(['OVERALL_ACCURACY', 'ACCURACY_WO_SPACES'], ascending =False).head(3))
#     print 'var'
#     display.display(by_smooth.var().sort_values(['OVERALL_ACCURACY', 'ACCURACY_WO_SPACES'], ascending =False).head(3))


def loop():
    
    # Copy and take subset
    char_inps = char_inps_raw[char_inps_raw.columns[:MAX_COLS]].copy()
    
    for _ in range(REPEATS):
        # Extract features
        for end in MFCC_END:
            for start in MFCC_START:
                for cluster in NUM_CLUSTERS:
                    for comp in N_COMPONENTS:
                        cepstrum_df = pl.extract_cepstrum(char_inps, RATE, mfcc_start=start, mfcc_end=end)

                        # Cluster the sounds
                        cepstrum_df = pl.cluster(cepstrum_df, num_clusters = cluster, n_components = comp)

                        # Repeat the hmm experiement for each parameter

                        # For each smoothing parameter to be tested
                        for smooth in SMOOTHS:


                            estimate, acc, acc_wospace, score, hmm = pl.run_hmm(cepstrum_df, targ_s, 
                                                                    cluster, t_smooth = smooth, tol = TOL,
                                                                    do_all = DO_ALL, verbose = False)  
                            print smooth, acc, acc_wospace, score
                            params,_id = build_params(smooth, end, start, cluster, comp)
                            params['OVERALL_ACCURACY'] = acc
                            params['ACCURACY_WO_SPACES'] = acc_wospace
                            params['PREDICTION'] = estimate
                            params['SCORE'] = score
                            params['MODEL'] = hmm

                            if acc > SAVE_THRESH:
                                save(hmm, acc, _id)
                            else:
                                save()
if should_loop:
    loop()

1.0 0.220967274833 0.124140427072 -13646.2184504
2.71828182846 0.200115841297 0.112196887441 -13733.1453885
7.38905609893 0.180422820736 0.0937386898299 -13577.2825122
20.0855369232 0.247610773241 0.169381107492 -13546.0075913
54.5981500331 0.15146249638 0.0586319218241 -13711.7439653
148.413159103 0.25166521865 0.177343467246 -13616.6269812
1.0 0.274254271648 0.205935577271 -13584.7949438
2.71828182846 0.198088618593 0.120883098082 -13544.9115054
7.38905609893 0.258615696496 0.188925081433 -13691.0412076
20.0855369232 0.205039096438 0.117987694535 -13775.3532427
54.5981500331 0.293657688966 0.232356134636 -13707.0363725
148.413159103 0.201563857515 0.119435396308 -13753.5025525
1.0 0.195482189401 0.116178067318 -13540.5494681
2.71828182846 0.155227338546 0.0550126673905 -13692.8357955
7.38905609893 0.183898059658 0.116901918205 -13567.1286493
20.0855369232 0.136403127715 0.038364096996 -13693.4311296
54.5981500331 0.194034173183 0.118711545422 -13532.7412975
148.413159103 0.1421951925

In [ ]:
save()

# Visualizing process 

In [ ]:
char_inps = char_inps_raw.copy()
char_inps = char_inps_raw[char_inps_raw.columns[:MAX_COLS]].copy()
char_inps.tail(10)

## Visualize Spaces 

In [ ]:
pl.view_char(char_inps, ' ', limit = 8, xlim = 600)

In [ ]:
pl.view_char(char_inps, 'e', limit = 8, xlim = 1000)

## Get the cepstrum features from the raw signal 

In [ ]:
 # Extract features
cepstrum_df = pl.extract_cepstrum(char_inps, RATE, mfcc_start=MFCC_START, mfcc_end=MFCC_END)

cepstrum_df.head()

In [ ]:
inds = cepstrum_df.dtypes == np.float64
data = cepstrum_df.ix[:,inds].values
# Standardize data
if WHITEN:
    data = whiten(data)
        
pca = PCA(n_components=N_COMPONENTS)
pca.fit(data)
data = pca.transform(data)
pca.explained_variance_ratio_.sum()

## Visualize Cepstrum Features 

In [ ]:
pl.view_char(cepstrum_df, ' ', xlim = 200, limit = 12)

In [ ]:
pl.view_char(cepstrum_df, 'e', xlim = 200, limit = 12)

## Cluster the features

In [ ]:
# Cluster the sounds
cepstrum_df = pl.cluster(cepstrum_df, num_clusters = NUM_CLUSTERS, n_components = N_COMPONENTS)
cepstrum_df.head()

In [ ]:
pl.cluster_proportions(cepstrum_df, [' '] + map(lambda x : chr(x + ord('a')),  range(26)), num_clusters = NUM_CLUSTERS)

In [ ]:
cepstrum_df['Count'] = 1
scores = cepstrum_df[['char', 'cluster', 'Count']].groupby(['cluster', 'char']).count()
n = NUM_CLUSTERS 
fig, axes = plt.subplots((n-1)/4+1, 4, figsize=(24, n*3/2))

# Repeat for each coefficient
for cl in range(n):
    axe = axes[cl / 4][cl % 4]
   
    _ = scores.ix[cl].plot(kind='bar', ax = axe)
    _ = axe.set_title(cl)

## Build Transmission Matrix with bigrams

In [ ]:
pl = reload(pl)
A_df, n_unique, unique_chars, id_to_char, char_to_id  = pl.build_transmission_full()
A_df.head()

##  Check the proportion of spaces across the k means classes

In [ ]:
space_props= pl.get_props(cepstrum_df, NUM_CLUSTERS, ' ')
_  = space_props.plot(kind='bar', title = 'Space Proportions')

## Build emissions matrix

In [ ]:
Eta = pl.build_eta(cepstrum_df, unique_chars, NUM_CLUSTERS, do_all = DO_ALL)
Eta


## Get Unigram proportions

In [ ]:
char_counts = pl.get_char_counts(targ_s, unique_chars)
char_counts

## Run HMM

In [ ]:
Eta

In [ ]:
# smooth = 0.5
# estimate, acc, acc_wospace, score, hmm = pl.run_hmm(cepstrum_df, targ_s, 
#                                                     NUM_CLUSTERS, t_smooth = smooth, tol = TOL,
#                                                     do_all = DO_ALL, verbose = True)  
# params,_id = build_params(smooth)
# params['OVERALL_ACCURACY'] = acc
# params['ACCURACY_WO_SPACES'] = acc_wospace
# params['PREDICTION'] = estimate
# params['SCORE'] = score
# params['MODEL'] = hmm

# if acc > SAVE_THRESH:
#     save(hmm, acc, _id)
# else:
#     save()

In [ ]:
# estimate

In [ ]:
targ_s

In [ ]:
# smooths = map(lambda x : np.exp(x), range(-10, 10))
# smooths = np.arange(0,2,.1)
for smooth in smooths:
    estimate, acc, acc_wospace, score, hmm = pl.run_hmm(cepstrum_df, targ_s, 
                                                    NUM_CLUSTERS, t_smooth = smooth, tol = TOL,
                                                    do_all = DO_ALL, verbose = False)  
    params,_id = build_params(smooth)
    params['OVERALL_ACCURACY'] = acc
    params['ACCURACY_WO_SPACES'] = acc_wospace
    params['PREDICTION'] = estimate
    params['SCORE'] = score
    params['MODEL'] = hmm

    if acc > SAVE_THRESH:
        save(hmm, acc, _id)
    else:
        save()
    print smooth, acc, acc_wospace, score
    print estimate

In [ ]:
def scrapeone():
    webpage = 'https://www.gutenberg.org/files/11/11-h/11-h.htm'
    html = urllib.urlopen(webpage).read()
    soup = BeautifulSoup(html, 'html.parser')
    texts = soup.findAll(text=True)
    import re
    def visible(element):
        if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
            return False
        elif re.match('<!--.*-->', str(element.encode('utf-8'))):
            return False
        return True

    unique_chars = [' '] + map(lambda x : chr(x + ord('a')), range(26))

    raw_text = filter(lambda x : x in unique_chars, ' '.join(filter(visible, texts)).replace('\r\n', ' ').replace('\n', ' ').replace('\u2018', '').replace('\u2019', '').lower())
    raw_text = ' '.join(raw_text.split())

def scrape():
    results = {}
    raw_text_all = ''
    webpages = ['https://www.gutenberg.org/files/11/11-h/11-h.htm', 'https://www.gutenberg.org/files/1342/1342-h/1342-h.htm',
               'https://www.gutenberg.org/files/46/46-h/46-h.htm', 'https://www.gutenberg.org/files/84/84-h/84-h.htm',
               'https://www.gutenberg.org/files/76/76-h/76-h.htm', 'https://www.gutenberg.org/files/844/844-h/844-h.htm',
               'https://www.gutenberg.org/files/53638/53638-h/53638-h.htm', 'https://www.gutenberg.org/files/2542/2542-h/2542-h.htm',
               'https://www.gutenberg.org/files/1400/1400-h/1400-h.htm', 'https://www.gutenberg.org/files/98/98-h/98-h.htm',
               'https://www.gutenberg.org/files/74/74-h/74-h.htm', 'https://www.gutenberg.org/files/53641/53641-h/53641-h.html',
               'https://www.gutenberg.org/files/1232/1232-h/1232-h.htm', 'https://www.gutenberg.org/files/1661/1661-h/1661-h.htm',
               'https://www.gutenberg.org/files/345/345-h/345-h.htm', 'https://www.gutenberg.org/files/160/160-h/160-h.htm',
               'https://www.gutenberg.org/files/5200/5200-h/5200-h.htm', 'http://www.gutenberg.org/cache/epub/30254/pg30254.html',
               'https://www.gutenberg.org/files/1952/1952-h/1952-h.htm', 'https://www.gutenberg.org/files/2600/2600-h/2600-h.htm',
               'https://www.gutenberg.org/files/174/174-h/174-h.htm', 'https://www.gutenberg.org/files/2701/2701-h/2701-h.htm']
    for page in tqdm(webpages):
        html = urllib.urlopen(webpage).read()
        soup = BeautifulSoup(html, 'html.parser')
        texts = soup.findAll(text=True)
        import re
        def visible(element):
            if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
                return False
            elif re.match('<!--.*-->', str(element.encode('utf-8'))):
                return False
            return True

        unique_chars = [' '] + map(lambda x : chr(x + ord('a')), range(26))

        raw_text = filter(lambda x : x in unique_chars, ' '.join(filter(visible, texts)).replace('\r\n', ' ').replace('\n', ' ').replace('\u2018', '').replace('\u2019', '').lower())
        raw_text = ' '.join(raw_text.split())
        results[page] = raw_text
        raw_text_all += raw_text
        time.sleep(2)
# with open('data/guttenberg_text.txt', 'w') as f:
#     f.write(raw_text_all)